# By using a the API Key from openweatherAPI, I am extracting the weather forecast for the next 3 days every 5 hours. The location data is being retrived from the Cities table created in the step beforehand 

In [1]:
#Libraries I used
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from sqlalchemy import create_engine 
import mysql.connector

## Setting up open weather API JSON file to decide which info to keep

In [2]:
# Berlin # testing first with one city to make sure it works and to determine what to keep from JSON
latitude = 52.520008
longitude = 13.404954

API_key = "89b4a954f2f955292f6b3136080e5184"

url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")

response = requests.get(url)
json_berlin = response.json()

json_berlin

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1745593200,
   'main': {'temp': 16.82,
    'feels_like': 15.42,
    'temp_min': 16.73,
    'temp_max': 16.82,
    'pressure': 1019,
    'sea_level': 1019,
    'grnd_level': 1014,
    'humidity': 33,
    'temp_kf': 0.09},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 27},
   'wind': {'speed': 4.67, 'deg': 55, 'gust': 5.22},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2025-04-25 15:00:00'},
  {'dt': 1745604000,
   'main': {'temp': 15.51,
    'feels_like': 14.19,
    'temp_min': 14.84,
    'temp_max': 15.51,
    'pressure': 1020,
    'sea_level': 1020,
    'grnd_level': 1015,
    'humidity': 41,
    'temp_kf': 0.67},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 49},
   'wind': {'speed': 4.53, 'deg': 27, 'gust': 7.01},
   'visibi

In [ ]:
# 🔧 CONFIG
schema = "gans_cloud"
host = "TOBEREPLACEDWITHIP"
user = "root"
password = "INSERTPASSWORDHERE"
port = 3306
API_key = "APIKEYFROMOPENWEATHERAPI"

In [7]:

# ✅ Step 1: Define database connection

# Create connection string for SQLAlchemy
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{schema}'


# ✅ Step 2: Load city data from database
cities_df = pd.read_sql("SELECT * FROM Cities", con=connection_string)


# Prepare list to hold all weather entries
weather_data = []

# ✅ Step 3: Loop over cities and fetch weather data
for _, city in cities_df.iterrows():
    latitude = city['Latitude']
    longitude = city['Longitude']
    city_id = city['City_id'] 

    # Make API call
    url = (
        f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}"
        f"&lon={longitude}&appid={API_key}&units=metric"
    )
    response = requests.get(url)
    data = response.json()

    # ✅ Step 4: Process weather entries for each city
    for entry in data['list']:
        weather_entry = {
            'city_id': city_id,
            'datetime': entry['dt_txt'],
            'temp': entry['main']['temp'],
            'feels_like': entry['main']['feels_like'],
            'humidity': entry['main']['humidity'],
            'weather_main': entry['weather'][0]['main'],
            'weather_desc': entry['weather'][0]['description'],
            'wind_speed': entry['wind']['speed'],
            'clouds': entry['clouds']['all'],
            'rain_3h': entry.get('rain', {}).get('3h', 0.0)
        }
        weather_data.append(weather_entry)

# ✅ Step 5: Create DataFrame
df = pd.DataFrame(weather_data)

# Convert datetime column
df['datetime'] = pd.to_datetime(df['datetime'])

# Capitalize column names
df.columns = df.columns.str.capitalize()

# ✅ Optional: Save to database
df.to_sql('Weather', con=connection_string, if_exists='append', index=False)


120

# SUCCESS! Now the table on SQL is filled with weather data for each City 